In [2]:
# install PyAudio http://people.csail.mit.edu/hubert/pyaudio/
# pip install pyaudio
import pyaudio
import wave
import sys

# default settings for record and playback
CHUNK = 1024
FORMAT = pyaudio.paInt16
CHANNELS = 1
RATE = 22050

def play_stream(wave_stream):
    global CHUNK
    print("Playing...")
    p = pyaudio.PyAudio()
    out_stream = p.open(format=p.get_format_from_width(wave_stream.getsampwidth()),
                channels=wave_stream.getnchannels(),
                rate=wave_stream.getframerate(),
                output=True)
    data = wave_stream.readframes(CHUNK)
    while data:
        out_stream.write(data)
        data = wave_stream.readframes(CHUNK)
    out_stream.stop_stream()
    out_stream.close()
    p.terminate()
    print("played")


def record_to_file(filename, seconds=5):
    global FORMAT, CHANNELS, RATE
    RECORD_SECONDS = seconds
    p = pyaudio.PyAudio()
    stream = p.open(format=FORMAT,
                channels=CHANNELS,
                rate=RATE,
                input=True,
                frames_per_buffer=CHUNK)
    print("Start recording... ")
    frames = []
    for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
        data = stream.read(CHUNK)
        frames.append(data)
    print("...recorded", seconds, "second(s)")
    stream.stop_stream()
    stream.close()
    p.terminate()
    wf = wave.open(filename, 'wb')
    wf.setnchannels(CHANNELS)
    wf.setsampwidth(p.get_sample_size(FORMAT))
    wf.setframerate(RATE)
    wf.writeframes(b''.join(frames))
    wf.close()

In [3]:
# yandex speech kit
api_key = "" #TODO your key

In [4]:
import requests
import uuid
from bs4 import BeautifulSoup

# this is unique idenifier of user
# API assumes that APPLICATION makes requests and we can mark them
UUID = str(uuid.uuid4()).replace("-", "")

def ask_yandex(wave_file, uid, lang="en-US"):
    # API doc
    # https://tech.yandex.ru/speechkit/cloud/doc/guide/concepts/asr-http-request-docpage/
    url = "https://asr.yandex.net/asr_xml?uuid={}&key={}&topic={}&lang={}&disableAntimat={}"
    url = url.format(uid, api_key, "queries", lang, "true")
    
    # just read raw information of the container
    data = open(wave_file, "rb").read()
    headers = {'Content-Type' : 'audio/x-wav', 'Content-Length' : str(len(data))}
    
    # do post request of data
    resp = requests.post(url, data=data, headers=headers)
    
    # parse answers
    dom = BeautifulSoup(resp.text, "lxml")    
    result = dict((var.string, float(var['confidence']))   
                  for var 
                  in dom.html.body.recognitionresults.findAll("variant"))
    return result

In [7]:
sample = "C:/dev/peterpiper.wav"
voice = "C:/dev/voice.wav"

# recognize generated speech
res = ask_yandex(sample, UUID)
print(*res.items(), sep="\n")

# recognize your speech
record_to_file(voice, 3)
wf = wave.open(voice, 'rb')
res = ask_yandex(voice, UUID)
print(*res.items(), sep="\n")

('Peter piper picked a peck of pickled peppers how many pickled peppers did peter piper PIC', 0.55)
('Peter piper picked a peck of pickled peppers how many pickled peppers did peter piper picked', -1000.0)
('Peter piper picked a peck of pickled peppers how many pickled peppers did peter piper pick', -1000.0)
('Peter piper picked a peck of pickled peppers howmany pickled peppers did peter piper PIC', -1000.0)
('Peter piper picked a peck of pickled peppers how many pickled peppers to peter piper PIC', -1000.0)
Start recording... 
...recorded 3 second(s)
('Where is my fish', 0.83)
('Where is my IP', 8.6)
('Where is my', 8.6)
('Where is my PC', 8.6)
('Where is my beach', 8.6)
